# MDA 2021
## Pyspark Sample Code
-----------------------------------------------------------------

## Setup
--------------------------------------------------

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

You should consider upgrading via the '/home/amin/.local/share/virtualenvs/HW-QXNApxFg/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/amin/.local/share/virtualenvs/HW-QXNApxFg/bin/python -m pip install --upgrade pip' command.
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


Now we authenticate a Google Drive client to processing data

**Make sure to follow the interactive instructions.**

In [2]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


## Check and extract data
--------------------------------------------------

In [ ]:
!ls '/content/drive/My Drive/Colab Notebooks/MDA/HW3/'

 checkstatus.csv   MDA_2021.ipynb       SystemID.csv
 CompanyID.csv	   Sample_Data.zip      Traffic.csv
 Data.zip	   Sample_Traffic.csv  'پروژه MDA2021.pdf'


In [2]:
# !unzip "/content/drive/My Drive/Colab Notebooks/MDA/HW3/Sample_Data.zip" -d "/content/drive/My Drive/Colab Notebooks/MDA/HW3"
!unzip "./Sample_Data.zip" -d "./"

Archive:  ./Sample_Data.zip
  inflating: ./Sample_Traffic.csv    


the cells above, extract data which is in '/content/drive/My Drive/Test' to /content/drive/My Drive/Test/Traffic.csv  

## Initializing Spark and read data
--------------------------------------------------

In [1]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,TimestampType
from pyspark.sql.functions import col,current_timestamp,to_date,hour,dayofweek
spark = SparkSession \
    .builder \
    .appName("Spark_Processor") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

schema = StructType([ \
        StructField("DEVICE_CODE", IntegerType(), True), 
        StructField("SYSTEM_ID",IntegerType(),True), \
        StructField("ORIGINE_CAR_KEY",StringType(),True), \
        StructField("FINAL_CAR_KEY",StringType(),True), \
        StructField("CHECK_STATUS_KEY", IntegerType(), True), \
        StructField("COMPANY_ID", StringType(), True), \
        StructField("PASS_DAY_TIME", TimestampType(), True)
    ])

22/01/26 08:58:53 WARN Utils: Your hostname, amin-X556UQK resolves to a loopback address: 127.0.1.1; using 192.168.1.15 instead (on interface wlp3s0)
22/01/26 08:58:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/26 08:58:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h3 dir="rtl">
 در این قسمت، لطفا آدرس فایل 
csv
 را به جای آدرس قرار داده شده جایگذاری کنید.  
  
</h3> 

In [2]:
# df=spark.read.csv('/content/drive/My Drive/Colab Notebooks/MDA/HW3/Sample_Traffic.csv',header=True,schema=schema)
df=spark.read.csv('Sample_Traffic.csv',header=True,schema=schema)
# df.select('DEVICE_CODE').distinct().count()
df.show(5)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|     200501|       81|       10477885|     10477885|               5|       161|2021-06-01 03:54:39|
|        155|       81|       87625017|     87625017|               5|       161|2021-06-01 04:14:21|
|     631757|       81|        8652928|      8652928|               5|       161|2021-06-01 03:58:57|
|     631757|       81|        8548123|      8548123|               5|       161|2021-06-01 04:01:38|
|     631757|       81|       24715264|     24715264|               5|       161|2021-06-01 03:56:57|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
only showing top 5 rows



<h3 dir="rtl">
 با استفاده از یک بار فرایند 
 map
 ، هر ردیف از 
 df
 را به صورت کلید پلاک خودرو، و زمان عبور از مسیر در می‌آوریم. مقدار این کلید را نیز برابر کد دستگاه (
     که نمایانگر مسیر عبوری است 
 ) 
 می‌گذاریم. 
 rdd
 به دست آمده به ازای هر مسیر در هر روز، یک ردیف خواهد داشت. 
</h3> 


In [3]:
rdd = df.rdd.map(lambda x: ((x['FINAL_CAR_KEY'], x['PASS_DAY_TIME'].date()),  x['DEVICE_CODE']))
rdd.take(5)

[(('10477885', datetime.date(2021, 6, 1)), 200501),
 (('87625017', datetime.date(2021, 6, 1)), 155),
 (('8652928', datetime.date(2021, 6, 1)), 631757),
 (('8548123', datetime.date(2021, 6, 1)), 631757),
 (('24715264', datetime.date(2021, 6, 1)), 631757)]

<h3 dir="rtl">
 اگر 
 rdd
 بالا را به کمک کلیدش 
 group_by
 کنیم، برای هر ماشین در هر روز خاص، تمام مسیر‌هایی که عبور کرده در یک لیست خواهیم داشت. البته به دلیل ماهیت مسئله 
 frequent_itemsets
 ، 
 به جای لیست از
 set
 استفاده کرده‌ام، تا مسیر‌های تکراری نداشته باشیم. هرچند می‌توان مسئله را به گونه‌ای تعریف کرد که ترتیب و یا تعداد تکرار مسیرها تاثیر گذار باشد، اما با توجه به خروجی‌ای که ما می‌خوایم، این فرض چیزی از کلیت مسئله نمی‌کاهد.  
</h3> 


In [31]:
path_rdd = rdd.groupByKey().mapValues(set)
path_rdd.take(5)

[(('69939810', datetime.date(2021, 6, 1)), {206602, 631830, 900233}),
 (('11046172', datetime.date(2021, 6, 1)), {206602}),
 (('29077699', datetime.date(2021, 6, 1)),
  {119,
   128,
   206602,
   208602,
   210602,
   631367,
   631763,
   631829,
   900135,
   900233,
   900234,
   900240,
   900246,
   900266,
   22010111,
   22010123}),
 (('40682798', datetime.date(2021, 6, 1)), {206602}),
 (('48823778', datetime.date(2021, 6, 1)), {206602})]

In [48]:
import numpy as np
from numpy import linalg as LA
import math

A = np.array([
    [0,1,1,0,0,0],
    [0,0,1,0,0,0],
    [1,1,0,1,1,1],
    [0,1,0,0,0,0],
    [0,1,0,0,0,1],
    [0,1,1,0,0,0],
])
M = np.matmul(A.T,A)
N = np.matmul(A,A.T)

m = LA.eig(M)
m

hubs = m[1][0]
norm2 = 0
for i in hubs:
    norm2 += i**2
print(hubs)
print(norm2)
hubs = hubs/ math.sqrt(norm2)
hubs
# LA.eig(N)
# LA.svd(M)


[-0.23922592 -0.32270464  0.37559075 -0.17577712 -0.09982089  0.8145577 ]
1.0068017862460348


array([-0.23841647, -0.32161272,  0.37431988, -0.17518235, -0.09948313,
        0.81180153])